<a href="https://colab.research.google.com/github/SterlingHayden/Pyomo-Introduction/blob/main/IntroductionToPyomo_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 3: Introduction to Pyomo

## Objectives

Students will be skilled at
1.   building small-scale concrete models in Pyomo
2.   solving small-scale concrete models using an appropriate solver

## References

1. [Pyomo Documentation](https://pyomo.readthedocs.io/en/stable/)
2. [CBC Documentation](https://projects.coin-or.org/Cbc)

## Overview of Pyomo and Solvers

Pyomo is a Python-based, open source optimization modeling language.  It supports a variety of solvers that can be used to solve different classifications of optimization models.  

We will use the following solvers in this class:

* **CBC**: Linear programs and mixed integer linear programs
* **Ipopt**: Nonlinear programs (possibly nonconvex)
* **Bonmin**: Mixed integer nonlinear programs with convex continuous relaxation
* **Couenne**: Mixed integer nonlinear programs

The following references provide installation instructions for each of the solvers:

1.   [Course Wiki](http://dascwiki.ddns.uark.edu/mediawiki/index.php/DASC_3203:_Optimization_Methods_in_Data_Science)
2.   [ND Pyomo Cookbook](https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html)


## Installing Pyomo and CBC

In [ ]:
!pip install -q pyomo

In [ ]:
!apt-get install -y -qq coinor-cbc

## Example 1: Solving a Linear Program Using Pyomo and CBC

### Problem Description (Taha, 2003)

Reddy Mikks produces both interior and exterior paints from two raw materials, M1 and M2.  The following table provides the basic data of the problem:


| **Raw Material** | Tons raw material per ton of exterior paint   | Tons raw material per ton of interior paint | Daily availability (tons)|
|---|---|---|---|
| M1 | 6 | 4 | 24 |
| M2 | 1 | 2 | 6 |

Each ton of exterior paint produced nets a profit of \$5000 and each ton of interior paint produced nets a profit of \$4000.  A market survey indicates the daily demand for interior paint cannot exceed that of exterior paint by more than 1 ton.  Also, the maximum daily demand of interior paint is 2 tons.  Reddy Mikks wants to determine the mix of interior and exterior paints that maximizes the total daily profit.







### Linear Programming Formulation

**Decision Variables**

* $x_1 = $ tons of exterior paint produced daily

* $x_2 = $ tons of interior paint produced daily

**Model**

$\begin{align}
\max \  && 5x_1 && + 4x_2 & & \textrm{(daily profit in \$1000s)}\\
\textrm{s.t.}\ && 6x_1 && + 4x_2 & \leq 24 & \textrm{(daily M1 availability in tons)}\\
&&   x_1 && + 2x_2 & \leq 6 & \textrm{(daily M2 availability in tons)} \\
&& -x_1 && +x_2 & \leq 1 & \textrm{(market survey)}\\
&& && x_2 & \leq 2 & \textrm{(maximum demand of interior paint)}\\
&& x_1, && x_2 & \geq 0
\end{align}$



### Building a *Concrete* Pyomo Model

A *Concrete* Pyomo model is a model in which coefficients (i.e., data) are fed to the model as the objective and constraints are defined.

In [ ]:
#Import Pyomo and define a concrete model
import pyomo.environ as pyo
model = pyo.ConcreteModel()

#Add variables x[1] and x[2] to model
model.x = pyo.Var([1,2], domain=pyo.NonNegativeReals)

#Define the objective "5x[1] + 4x[2]"
model.objective = pyo.Objective(expr= 5*model.x[1] + 4*model.x[2], sense=pyo.maximize)

#Define the constraints
model.constraint1 = pyo.Constraint(expr = 6*model.x[1] + 4*model.x[2] <= 24)
model.constraint2 = pyo.Constraint(expr = model.x[1] + 2*model.x[2] <= 6)
model.constraint3 = pyo.Constraint(expr = -model.x[1] + model.x[2] <= 1)
model.constraint4 = pyo.Constraint(expr = model.x[2] <= 2)


### Printing the model

In [ ]:
model.pprint()

### Solving the concrete model

In [ ]:
#Declare the solver as CBC
opt = pyo.SolverFactory('cbc')

#Solve the model
result = opt.solve(model)
result.write()

### Retrieving the results

In [ ]:
#Access and print the variable values
print("x[1] = ", pyo.value(model.x[1]))
print("x[2] = ", pyo.value(model.x[2]))

#Access and print the objective value
print("Objective Value = ", pyo.value(model.objective))

In [ ]:
#Access and print the slack of each constraint
#Use "uslack" because they were <= constraints
#Use "lslack" if they are >= constraints
print("constraint 1 slack = ", pyo.value(model.constraint1.uslack()))
print("constraint 2 slack = ", pyo.value(model.constraint2.uslack()))
print("constraint 3 slack = ", pyo.value(model.constraint3.uslack()))
print("constraint 4 slack = ", pyo.value(model.constraint4.uslack()))